## Model for Saliency

###### adapted ImacheCache from nnvision and monkey_loaders from nnvision.datasets



In [74]:
%load_ext autoreload
%autoreload 

import datajoint as dj
dj.config['enable_python_native_blobs'] = True


dj.config['database.user']= 'kanderer'
dj.config['database.password']= 'enamel-vendetta-deodorant'


schema_name = 'nnfabrik_monkey_saliency'

schema = dj.schema(schema_name, locals())
dj.config['nnfabrik.schema_name'] = schema_name


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import torch

import nnfabrik
from nnfabrik import builder


import numpy as np
import pickle
import os

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt

import nnvision

from nnvision.utility.measures import get_correlations

# NNfabrik intro: Using the builder to build the dataloader objects, models, trainer

In [57]:
# here's where the data is on the server:
os.listdir('/data')

['image_classification', 'mouse', 'monkey', 'fetched_from_attach']

In [58]:
#### loading monkey data

basepath = '/data/monkey/toliaslab/CSRF19_V4'
neuronal_data_path = os.path.join(basepath, 'neuronal_data/')
neuronal_data_files = [neuronal_data_path + f for f in listdir(neuronal_data_path) if isfile(join(neuronal_data_path, f))]
image_cache_path = os.path.join(basepath, 'images')

saliency_cache_path = os.path.join(basepath, 'images_saliency')
print(saliency_cache_path)

/data/monkey/toliaslab/CSRF19_V4/images_saliency


# Part Two: NNfabrik and DataJoint

Instead of using the builder to get the data/model/and trainer, we can use datajoint to manage that process for us.
There are Model, Dataset, and Trainer Tables. And each combination in those tables should in principle lead to a fully trained model.
For completeness, there is also a Seed table that stores the random seed, and a Fabrikant table, that stores the name and contact details of the creator (=Fabrikant).


### Make sure that a dj-database is connected. Recommended dj version is 0.12.4

In [59]:
import datajoint as dj

dj.config['enable_python_native_blobs'] = True

from nnfabrik.templates.trained_model import TrainedModelBase
from nnfabrik.main import *
import os

if not 'stores' in dj.config:
    dj.config['stores'] = {}
    
dj.config['stores']['minio'] = {  # store in s3
    'protocol': 's3',
    'endpoint': os.environ.get('MINIO_ENDPOINT', 'DUMMY_ENDPOINT'),
    'bucket': 'nnfabrik',
    'location': 'dj-store',
    'access_key': os.environ.get('MINIO_ACCESS_KEY', 'FAKEKEY'),
    'secret_key': os.environ.get('MINIO_SECRET_KEY', 'FAKEKEY')
}

In [60]:
a = Dataset.full_table_name

In [61]:
Dataset.database

'nnfabrik_monkey_saliency'

In [8]:
# change this entry to reflect your datajoint username
Fabrikant().insert1(dict(fabrikant_name='kanderer',
                         email="k.anderer@t-online.de",
                         affiliation='sinzlab',
                         dj_username="kanderer"))

DuplicateError: ("Duplicate entry 'kanderer' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [62]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,full_name full name of the person,email e-mail address,affiliation conributor's affiliation (e.g. Sinz Lab),dj_username DataJoint username
kanderer,,k.anderer@t-online.de,sinzlab,kanderer


In [63]:
schema

Schema `nnfabrik_monkey_saliency`

In [64]:
Seed().insert([{'seed':1000}])
Seed()

DuplicateError: ("Duplicate entry '1000' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

###  add entries for dataset, model, and trainer, with their corresponding configurations

#### Dataset

In [52]:
# Specifying the dataset function: its defined in nnvision/datasets, and has to present in the __init__.py there.
# Specifying the dataset function: its defined in nnvision/datasets, and has to present in the __init__.py there.
dataset_fn = 'nnsaliency.datasets.saliency_loaders.monkey_saliency_loader'
dataset_config = {'dataset': 'CSRF_V4',
  'neuronal_data_files': ['/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3645713184967.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646146362280.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646222772876.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646321567215.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646401671909.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646657276979.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646747773987.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646924284383.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647439534447.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647531807425.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647617982355.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648051669051.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648221742562.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648475457626.pickle',
    '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3654087847388.pickle'],
  'image_cache_path': '/data/monkey/toliaslab/CSRF19_V4/images/',
     'saliency_cache_path': '/data/monkey/toliaslab/CSRF19_V4/images_saliency_scaled/',
  'crop': [(108, 0), (146, 146)],
  'subsample': 1,
  'scale': 0.4,
  'seed': 1000,
  'batch_size': 64,
    'image_frac': 0.8,
   'image_selection_seed': 1000,
  'time_bins_sum': (3, 4, 5, 6, 7, 8, 9, 10, 11),
                  'gradient': False,
                  'logarithm': True,
                  'include_all': False
                 }


In [53]:
# adds the dataset_function and dataset config that we defined above to the datase table
Dataset().add_entry(dataset_fn, dataset_config, dataset_comment='CSRF_V4, grad+sal+crop108, 146, small dataset')
#Dataset().fetch(as_dict=True)

{'dataset_fn': 'nnsaliency.datasets.saliency_loaders.monkey_saliency_loader',
 'dataset_hash': '472ef509aea9875112240a1b03d0fee6'}

About to delete:
`nnfabrik_monkey_saliency`.`dataset`: 1 items


Proceed? [yes, No]:  yes


Committed.


#### Model

In [20]:
model_fn = 'nnvision.models.se_core_full_gauss_readout'
model_config = {'pad_input': False,
  'gauss_type': 'isotropic',
  'gamma_readout': 0.43,
  'gamma_input': 10,
  'layers': 6,
  'depth_separable': True,
  'n_se_blocks': 0,
  'stack': -1,
  'input_kern': 15,
  'hidden_kern': 9,
  'hidden_channels': 32,
  'init_mu_range': 0.1780563002765732,
  'init_sigma': 0.5851238789405838}
#Model().add_entry(model_fn, model_config, model_comment='isotropic')
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nnvision.models.models.se_core_attention_readout,944ca98fa137d6edb15747d7517b9845,=BLOB=,kanderer,se_core_attention,2020-12-10 17:59:05
nnvision.models.models.se_core_saliency_shifted_readout,8e035a92630720888d6c55495c73d4b0,=BLOB=,kanderer,"saliency_shifted_readout, images for remapper",2020-12-20 22:30:11
nnvision.models.models.se_core_saliency_shifted_readout,b264a93e594b44d9744a82c291864016,=BLOB=,kanderer,"saliency_shifted_readout, no saliency!",2020-12-17 20:07:24
nnvision.models.models.se_core_saliency_shifted_readout,fcf83704246f464b2e071135cc039dd7,=BLOB=,kanderer,saliency_shifted_readout,2020-12-14 12:26:54
nnvision.models.ptrmodels.task_core_gauss_readout,07b08b47c3f1805168980920767c4e32,=BLOB=,kanderer,3channel_resnet,2020-12-02 11:11:03
nnvision.models.ptrmodels.task_core_gauss_readout,3ea73bf4504bc844e24d4fdf4a2728a9,=BLOB=,kanderer,isotropic,2020-12-01 21:16:58
nnvision.models.se_core_full_gauss_readout,a08c72eb2972056853de6f1ce881e0f0,=BLOB=,kanderer,isotropic,2020-11-30 11:04:53
nnvision.models.se_core_full_gauss_readout,cec63aa4435b2a205ec02eafc0a745ee,=BLOB=,kanderer,"isotropic, other config",2020-12-05 12:40:09
nnvision.models.se_core_saliency_shifted_readout,4d3353aec94d62b89e9aab1201ac66a1,=BLOB=,kanderer,"saliency shifter, only image, no remapper",2021-01-10 18:35:37
nnvision.models.se_core_saliency_shifted_readout,536fd5b36e93e087204dd7980ce61359,=BLOB=,kanderer,"saliency shifter, only image, only 1 channel for remapper",2020-12-30 16:40:50


In [21]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = dict(max_iter=5, 
                      lr_decay_steps=4, 
                      tolerance=0.0005, 
                      patience=5,
                      verbose=False, 
                      lr_init=0.0045,
                      avg_loss=False,
                      device='cuda')

#Trainer().add_entry(trainer_fn, trainer_config, trainer_comment="max_iter: 5")
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnvision.training.nnvision_trainer,0f139a9f3d68757b8104399a8425e4e8,=BLOB=,kanderer,max_iter: 5,2020-12-04 16:51:43
nnvision.training.nnvision_trainer,9263acaab930c44135472acbbf78339b,=BLOB=,kanderer,max_iter:100,2020-11-30 17:59:35
nnvision.training.nnvision_trainer,f03a6527ab0422767da50e67e2d543ef,=BLOB=,kanderer,"max_iter: 100, track",2020-12-05 20:30:10


####  The TrainedModel is a template, which can be found in nnfabrik.template.py

the trained model table is taking care of model training, and stores the model state in a part table. For further analyses of the trained model, one can either overwrite the TrainedModel definition by inheriting from the Base template class, or by attaching other tables to trained model.

In [40]:
# creating the simples TrainedModel class
print(type(TrainedModelBase))

@schema
class TrainedModel(TrainedModelBase):
    table_comment = "Trained models"
    storage = "minio"
    model_table = Model
    dataset_table = Dataset
    trainer_table = Trainer
    seed_table = Seed
    user_table = Fabrikant

<class 'datajoint.user_tables.OrderedClass'>


In [27]:
#TrainedModel.fetch(as_dict=True)

#dj.Diagram(schema)

as primary keys, it has the hashes of all the configurations, and it stores the score, and the output (which are defined in the respective trainer)

## Lets populate

In [83]:
key = dict(model_hash='8e035a92630720888d6c55495c73d4b0', dataset_hash='472ef509aea9875112240a1b03d0fee6', trainer_hash = 'f03a6527ab0422767da50e67e2d543ef')
TrainedModel().populate(key)

Loading images ...


Epoch 1: 100%|██████████| 2325/2325 [01:43<00:00, 22.45it/s]


[001|00/05] ---> 0.11710760742425919


Epoch 2: 100%|██████████| 2325/2325 [01:44<00:00, 22.27it/s]


[002|00/05] ---> 0.13446044921875


...

[065|03/05] -/-> 0.22703005373477936


Epoch 66: 100%|██████████| 2325/2325 [01:49<00:00, 21.29it/s]


[066|04/05] -/-> 0.2224852740764618


Epoch 67: 100%|██████████| 2325/2325 [01:49<00:00, 21.14it/s]


[067|05/05] -/-> 0.2263125479221344
Restoring best model after lr decay! 0.226313 ---> 0.227764
Restoring best model! 0.227764 ---> 0.227764


In [ ]:
TrainedModel()

In [ ]:
model_hash = TrainedModel().fetch1("model_hash")

In [ ]:
# now if you want to build the model again, we can use the .load_model() function of the trained model table.
# To use the load model function, the table needs to be restricted to one Entry. 
# for example: restricting with a key:
some_key = dict(model_hash=model_hash)
TrainedModel&some_key



# How to store Test Correlation

In [85]:
from nnvision.tables.scores import TestCorrelationScore
from nnvision.tables.from_nnfabrik import ScoringBaseNeuronType

@schema
class TestCorrelation(ScoringBaseNeuronType):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    measure_function = staticmethod(get_correlations)
    measure_dataset = "test"
    measure_attribute = "test_correlation"

TestCorrelation().populate(example_key, display_progress=True)

NameError: name 'MonkeyExperiment' is not defined

# How to Load a Model


In [77]:
#key = dict(model_hash='07b08b47c3f1805168980920767c4e32', dataset_hash='6cdb8f03fa4d0e752d6b2eb886c83e8c', trainer_hash = 'f03a6527ab0422767da50e67e2d543ef')

dataloader, model = (TrainedModel & key).load_model()

Loading images ...


In [78]:
# that is the trained model, with the state dict loaded and all. lets set to eval and start using it
model.eval();

In [79]:
from nnvision.utility.measures import get_correlations
test_correlation = get_correlations(model, dataloader["test"], device='cuda', as_dict=False, per_neuron=True)



In [81]:
np.mean(test_correlation)

0.21219248

# Parameter Extension

In [ ]:
# There's also the parameter extension, so that you can restrict with the config objects as well.

In [ ]:
from nnfabrik.utility.dj_helpers import create_param_expansion, make_definition
ModelExpanded = create_param_expansion('nnvision.models.se_core_full_gauss_readout', Model,fn_field='model_fn', config_field='model_config')
ModelParams = schema(ModelExpanded)

In [ ]:
ModelParams()

In [ ]:
ModelParams.populate()

In [ ]:
ModelParams()

In [ ]:
# for example:
Model*ModelParams&"hidden_kern=5"

In [ ]:
# Now you can just use that for building the model:
dataloaders, model =(TrainedModel & ((Model * ModelParams)&"hidden_kern=5")).load_model()